In [0]:
import http.server
import socketserver
import datetime
import sqlite3
import re




class RequestHandler(http.server.SimpleHTTPRequestHandler): # définition du nouveau handler, classe dérivée de http.server.SimpleHTTPRequestHandler

    static_dir = '/client' # sous-répertoire racine des documents statiques
    
    
    
    def do_GET(self): # on surcharge la méthode qui traite les requêtes GET    
        
        a=re.match('.*/country/(\w+).*',self.path) #trouve le pays correspondant dans une URL /country/pays
        
        exceptions=[self.path[0:5]!='/time',self.path[0:10]!='/countries',a==None]  #définit tous les cas d'utilisation non habituels       
        
        if not False in exceptions : #si la requete correspond à un cas d'utilisation classique
            self.path = self.static_dir + self.path  # on modifie le chemin d'accès en insérant un répertoire préfixe 
            http.server.SimpleHTTPRequestHandler.do_GET(self) # on traite la requête via la classe parent
           
        
        else : #si la requete ne correspond pas à un cas d'utilisation classique 
            
            self.temps() #affiche la page date/heure si la requete correspond 
            self.lister_pays()  #affiche une page contenant la liste des pays si l'URL correspond
            self.carac_pays_modif(a)   #affiche une page contenant les caractéristiques d'un pays si l'URL correspond
           
            
        
        
                
        
        
        
        
   
            
            
            
            
            
        
            
            

    def send(self,body,headers=[]):     
        
        encoded = bytes(body, 'UTF-8')  # on encode la chaine de caractères à envoyer, body est une string           
        self.send_response(200)   # on envoie la ligne de statut       
        [self.send_header(*t) for t in headers] # on envoie les lignes d'entête et la ligne vide
        self.send_header('Content-Length',int(len(encoded)))
        self.end_headers()        
        self.wfile.write(encoded) # on envoie le corps de la réponse
        
    
    
    
    
    def temps(self):  #affichage de la page date/heure si le path correspond 
        if self.path[0:5]=='/time':  #si le chemin se termine par '/time'
            affichage_date=datetime.datetime.today().ctime() #obtention de l'affichage de date/heure 
            
            response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Voici l'heure du serveur : {} GMT </p>"\
.format(affichage_date)
            self.send(response) 
            
    
    
    
    
    
    
    
    
    def lister_pays(self):  #affiche une page avec la liste des pays si le path correspond
        
        if self.path[0:10]=='/countries':
            conn=sqlite3.connect('pays.sqlite')  #connexion à la BD SQL
            c=conn.cursor()
            c.execute("SELECT name FROM countries")
            a=c.fetchall()
            liste_pays=[a[i][0] for i in range(len(a))]  #stocke la liste des pays            
            
            if  len(liste_pays)==0:  #s'il n'y a aucun pays dans la DB
                response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Pas de pays dans la database  </p>"
                self.send(response)              
                
                
            else: #s'il y a au moins un pays dans la DB
                        
                response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Liste des pays :  </p>" 
                
                
                for i in range(len(liste_pays)):  #ajoute le i-ème pays à la ligne suivante à afficher
                    response=response + \
"<meta charset='utf-8'><p>"+"["+str(i+1)+"] - "+liste_pays[i]+"</p>"                       
                self.send(response)
                
                
    def carac_pays(self,a):
        if a!=None:
            pays=a.group(1)  #récupère la string contenant le nom du pays dans l'URL           
            
            conn=sqlite3.connect('pays.sqlite')  #connexion à la base de données      
            c=conn.cursor()
                  
                  
            c.execute("SELECT name FROM sqlite_master WHERE type='table';")  #requête permettant d'avoir la liste des tables
            a=c.fetchall()  #stocke les résultats de la requête sous une forme alambiquée
            liste_tables=[a[i][0] for i in range(len(a))]  #stocke la liste des tables de la database (strings)
            
            if 'countries' in liste_tables : #vérifie que la table countries existe         
                c.execute("PRAGMA table_info(countries)")#requête permettant d'avoir la liste des colonnes de la table countries
                a=c.fetchall()   #stocke les résultats de la requête sous une forme alambiquée
                liste_colonnes=[a[i][1] for i in range(len(a))]  #stocke la liste des colonnes de la table countries (strings)    
            
                c.execute("SELECT * FROM countries WHERE wp="+"'"+pays+"'")
                a=c.fetchall()
                
                if len(a)!=0:  #vérifie que le pays existe dans la base données
                    caracteristiques_pays={liste_colonnes[i]:a[0][i] for i in range(len(liste_colonnes))}
                
                    response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Caractéristiques du pays :  </p>" 
                
                
                    for clef in caracteristiques_pays.keys():  #ajoute la caractéristique cllef à la ligne suivante de la réponse à afficher
                        response=response + \
"<meta charset='utf-8'><p>{} : {}  </p>"\
.format(clef,caracteristiques_pays[clef])                      
                    self.send(response)
                    
                
                else:  #Si le pays n'existe pas dans la base données
                    response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Ce pays n'est pas présent sur ce continent</p>"
                    self.send(response) 
                
                
    def carac_pays_modif(self,a):
        if a!=None:
            pays=a.group(1)  #récupère la string contenant le nom du pays dans l'URL           
            
            conn=sqlite3.connect('pays.sqlite')  #connexion à la base de données      
            c=conn.cursor()
                  
                  
            c.execute("SELECT name FROM sqlite_master WHERE type='table';")  #requête permettant d'avoir la liste des tables
            a=c.fetchall()  #stocke les résultats de la requête sous une forme alambiquée
            liste_tables=[a[i][0] for i in range(len(a))]  #stocke la liste des tables de la database (strings)
            
            if 'countries' in liste_tables : #vérifie que la table countries existe         
                c.execute("PRAGMA table_info(countries)")#requête permettant d'avoir la liste des colonnes de la table countries
                a=c.fetchall()   #stocke les résultats de la requête sous une forme alambiquée
                liste_colonnes=[a[i][1] for i in range(len(a))]  #stocke la liste des colonnes de la table countries (strings)    
            
                c.execute("SELECT * FROM countries WHERE wp="+"'"+pays+"'")
                a=c.fetchall()
                
                if len(a)!=0:  #vérifie que le pays existe dans la base données
                    caracteristiques_pays={liste_colonnes[i]:a[0][i] for i in range(len(liste_colonnes))}
                
                    response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><h1>{} </h1>"\
.format(caracteristiques_pays['name']) + \
"<ul>" +\
"<li>Continent : {}</li>"\
.format(caracteristiques_pays['continent'].capitalize())+\
"<li>Capital : {}</li>"\
.format(caracteristiques_pays['capital'].capitalize())+\
"<li>Latitude : {}</li>"\
.format(caracteristiques_pays['latitude'])+\
"<li>Longitude : {}</li>"\
.format(caracteristiques_pays['longitude'])+\
"</ul>"

                    self.send(response)
                    
                
                else:  #Si le pays n'existe pas dans la base données
                    response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Ce pays n'est pas présent sur ce continent</p>"
                    self.send(response)        
            
        
    
        
            
    

    
httpd = socketserver.TCPServer(("", 8080), RequestHandler)# instanciation et lancement du serveur
httpd.serve_forever()